In [5]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
from textblob import TextBlob
import spacy
import spacy.util
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, words as nltk_words
from collections import Counter
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
trans = preprocessing.MinMaxScaler()
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import string

In [6]:
nlp = spacy.load("en_core_web_lg")
#nlp = spacy.load("/kaggle/input/en-core-web-lg/en_core_web_lg/en_core_web_lg-3.8.0")

In [7]:
download_dir="./.venv/nltk_data"
#download_dir="/root/nltk_data"

In [8]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')
from tqdm import tqdm
# IN KAGGLE:
# train = pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/train.csv")
# test = pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")
display(train.head(2))

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


# Text Feature Engineering

In [9]:
def count_capital_words(text):
    return sum(map(str.isupper,text.split()))

In [10]:
var = 'n_capital'
train[var]=train['full_text'].apply(count_capital_words)
test[var]=test['full_text'].apply(count_capital_words)

## Punctuations

In [11]:
def count_punctuations(text):
    punctuations="'!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~'"
    count=0
    for i in punctuations:
        count+=text.count(i)
    return count

In [12]:
var = 'n_punct'
train[var]=train['full_text'].apply(count_punctuations)
test[var]=test['full_text'].apply(count_punctuations)

## Number of Unique Words

In [13]:
def n_unique_words(text):
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text.lower()) 
    return len(set(words))

In [14]:
var = 'n_unique'
train[var]=train['full_text'].apply(n_unique_words)
test[var]=test['full_text'].apply(n_unique_words)

Number of Unique Words (exclude stop words)

In [15]:
def n_unique_words_no_stop(text):
    text = text.lower().translate(str.maketrans("", "", string.punctuation))
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    unique_words = np.unique(words)
    return len(unique_words)

In [16]:
var = 'n_unique_n_stop'
train[var]=train['full_text'].apply(n_unique_words_no_stop)
test[var]=test['full_text'].apply(n_unique_words_no_stop)

In [17]:
def n_non_words(text):
    text = text.lower().translate(str.maketrans("", "", string.punctuation))
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words("english"))
    english_words = set(nltk_words.words())
    tokens = tokenizer.tokenize(text)
    non_words = [t for t in tokens if t not in stop_words and t not in english_words]
    return len(set(non_words))

In [18]:
var = 'n_n_word'
train[var]=train['full_text'].apply(n_non_words)
test[var]=test['full_text'].apply(n_non_words)

## Number of Noun Phrases

In [19]:
def count_noun_phrases(text):
    blob = TextBlob(text)
    return len(blob.noun_phrases)

In [20]:
var = 'noun_phrase_count'
train[var]=train['full_text'].apply(count_noun_phrases)
test[var]=test['full_text'].apply(count_noun_phrases)

## Parts of Speech Tagging

In [21]:
def pos(text):
    doc = nlp(text)
    result = dict(Counter([t.pos_ for t in doc]))
    missing = set(var) - set(result.keys())
    for miss in missing:
        result[miss] = 0.0
    return pd.Series(result)

In [22]:
print("Starting POS")
var = ['PRON', 'VERB',	'SCONJ', 'NOUN', 'AUX', 'ADP', 'PUNCT', 'PART',	'CCONJ', 'ADV', 'DET', 'ADJ', 'SPACE', 'PROPN', 'NUM', 'INTJ', 'SYM', 'X']
train[var]=train['full_text'].apply(pos)
test[var]=test['full_text'].apply(pos)
print("Finished POS")

Starting POS
Finished POS


### Sentiment Polarity

In [23]:
def essay_polarity(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [24]:
var = 'polarity'
train[var]=train['full_text'].apply(essay_polarity)
test[var]=test['full_text'].apply(essay_polarity)

### Essay Subjectivity

In [25]:
def essay_subjectivity(text):
    blob = TextBlob(text)
    return blob.sentiment.subjectivity

In [26]:
var = 'subjectivity'
train[var]=train['full_text'].apply(essay_subjectivity)
test[var]=test['full_text'].apply(essay_subjectivity)

### Spellchecker
score: 0 (no mistake) - 1 (all wrong)

In [27]:
def spell_similarity(text): 
    b = nlp(TextBlob(text).correct().string)
    return nlp(text).similarity(b)

In [28]:
tqdm.pandas()
print("Starting Spellchecker")
train['spell_score'] = train['full_text'].apply(spell_similarity)
test['spell_score'] = test['full_text'].apply(spell_similarity)
print("Finished Spellchecker")

Starting Spellchecker


KeyboardInterrupt: 

### Sentence Statistics

In [ ]:
def sentence_lengths(text):
    sentences = sent_tokenize(text)
    lengths = [n_unique_words(sent) for sent in sentences if sent.strip()]
    return lengths if lengths else [0]

In [ ]:
def sentence_av_len_calc(text):
    return np.mean(sentence_lengths(text))

train['av_sent_len']=train['full_text'].apply(sentence_av_len_calc)
test['av_sent_len']=test['full_text'].apply(sentence_av_len_calc)

In [ ]:

def sentence_max_len_calc(text):
    return np.max(sentence_lengths(text))

train['max_sent_len']=train['full_text'].apply(sentence_max_len_calc)
test['max_sent_len']=test['full_text'].apply(sentence_max_len_calc)

In [ ]:
def sentence_min_len_calc(text):
    return np.min(sentence_lengths(text))

train['min_sent_len']=train['full_text'].apply(sentence_min_len_calc)
test['min_sent_len']=test['full_text'].apply(sentence_min_len_calc)

In [ ]:
def sentence_median_len_calc(text):
    return np.median(sentence_lengths(text))

train['med_sent_len']=train['full_text'].apply(sentence_median_len_calc)
test['med_sent_len']=test['full_text'].apply(sentence_median_len_calc)

In [ ]:
def sentence_std_len_calc(text):
    return np.std(sentence_lengths(text))

train['std_sent_len']=train['full_text'].apply(sentence_std_len_calc)
test['std_sent_len']=test['full_text'].apply(sentence_std_len_calc)

### Number of sentences

In [ ]:
def count_sentences(text):
    return len(nltk.sent_tokenize(text))

In [ ]:
train['num_sent']= train["full_text"].apply(count_sentences)
test['num_sent']=test["full_text"].apply(count_sentences)

### Sentiment Scores

In [ ]:
# nltk.download('vader_lexicon', download_dir=download_dir)

sentiment scores: –4 (very negativ) and +4 (very positiv).

In [ ]:
def generate_sentiment_scores(data):
    sid = SentimentIntensityAnalyzer()
    neg=[]
    pos=[]
    neu=[]
    comp=[]
    for sentence in tqdm(data['full_text'].values): 
        sentence_sentiment_score = sid.polarity_scores(sentence)
        comp.append(sentence_sentiment_score['compound'])
        neg.append(sentence_sentiment_score['neg'])
        pos.append(sentence_sentiment_score['pos'])
        neu.append(sentence_sentiment_score['neu'])
    return comp,neg,pos,neu
train['compound'],train['negative'],train['positive'],train['neutral']=generate_sentiment_scores(train)
test['compound'],test['negative'],test['positive'],test['neutral']=generate_sentiment_scores(test)

In [ ]:
display(train[['full_text', 'compound', 'negative', 'positive', 'neutral']].head(5))

### Character Length

In [ ]:
train['char_len']=train['full_text'].apply(lambda x:len(x.split()))
test['char_len']=test['full_text'].apply(lambda x:len(x.split()))

In [ ]:
processed_train = pd.DataFrame()
processed_test = pd.DataFrame()
processed_train["tokens"] = train["full_text"].apply(lambda s: word_tokenize(s))
processed_test["tokens"] = test["full_text"].apply(lambda s: word_tokenize(s))

### Word Diversity

How many different words contains the text?

In [ ]:
def lexical_diversity(tokens):
    return len(set(tokens)) / max(len(tokens), 1)

train["lex_diversity"] = processed_train["tokens"].apply(lexical_diversity)
test["lex_diversity"] = processed_test["tokens"].apply(lexical_diversity)

## Tf-idf

In [ ]:
tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=10_000,
    min_df=5
)
X_tfidf_train = tfidf.fit_transform(train['full_text'])
X_tfidf_test = tfidf.transform(test['full_text'])

In [ ]:
tfidf_means = np.asarray(X_tfidf_train.mean(axis=0)).flatten()
tfidf_features = tfidf.get_feature_names_out()

In [ ]:
# Convert to DataFrame
tfidf_df_train = pd.DataFrame(X_tfidf_train.toarray(), columns=tfidf_features, index=train.index)
tfidf_df_test = pd.DataFrame(X_tfidf_test.toarray(), columns=tfidf_features, index=test.index)

In [ ]:
# Concatenate to original DataFrames
train = pd.concat([train.reset_index(drop=True), tfidf_df_train.reset_index(drop=True)], axis=1)
test = pd.concat([test.reset_index(drop=True), tfidf_df_test.reset_index(drop=True)], axis=1)

### Extract the labels

In [ ]:
y_train=train[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']]

# Load feature engineered data

In [ ]:
train.to_csv('data/feature_engineered/train_vars_test.csv', index=False)
y_train.to_csv('data/feature_engineered/y_vars_test.csv', index=False)
test.to_csv('data/feature_engineered/test_vars_test.csv', index=False)
#train.to_csv('/kaggle/working/train_vars.csv', index=False)
#y_train.to_csv('/kaggle/working/train_y_vars.csv', index=False)
#test.to_csv('/kaggle/working/test_vars.csv', index=False)

In [ ]:
display(train.head(5))

In [ ]:
df_train = pd.read_csv("data/feature_engineered/train_vars_test.csv")
df_test = pd.read_csv("data/feature_engineered/test_vars_test.csv")
#df_train = pd.read_csv("/kaggle/working/train_vars.csv")
#df_test = pd.read_csv("/kaggle/working/test_vars.csv")
display(df_train.head(3))

In [ ]:
label_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

In [ ]:
feature_cols = df_train.columns.difference(['text_id', 'full_text'] + label_cols).tolist()

In [ ]:
df_train = df_train.dropna()
df_test = df_test.dropna()

## Loss function

In [ ]:
class MCRMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')

    def forward(self, preds, targets):
        colwise_mse = self.mse(preds, targets).mean(dim=0)
        colwise_rmse = torch.sqrt(colwise_mse + 1e-8)
        return colwise_rmse.mean()

# Model training

In [ ]:
X = df_train[feature_cols]
y_all = df_train[label_cols]

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_all, test_size=0.2, random_state=42
)

# train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# convert in torch.Tensor 
y_pred_tensor = torch.tensor(y_pred, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32) 

# calculate MCRMSE 
mcrmse_loss = MCRMSELoss()
loss_value = mcrmse_loss(y_pred_tensor, y_test_tensor)

print(f"Evaluation mit MCRMSE: {loss_value.item():.4f}")

In [ ]:
# X_submission = df_test[feature_cols]

# submission_preds = model.predict(X_submission)

# submission_df = pd.DataFrame(submission_preds, columns=label_cols)
# submission_df["text_id"] = df_test["text_id"] 
# submission_df = submission_df[["text_id"] + label_cols]  

# submission_df.to_csv("submission.csv", index=False)